# SELLER P&L ANALYSIS

# Problem statement

> Should Olist remove underperforming sellers from its marketplace?

To analyse the impact of removing the worse sellers from Olist's marketplace, we can start with a what-if analysis: What would have happened if Olist had never accepted these sellers in the first place? For that:

In [51]:
# Pseudo-code

#1. Total Revenue = Sales * 10% + Months_on_olist * 80 BRL
#2. Review_cost = Mapping of reviews and IT costs
#3. Calculate profit per seller
#4. Analyse results

In [123]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from olist.seller import Seller
sellers = Seller().get_training_data()
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,review_cost,n_orders,quantity,quantity_per_order,sales
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,1.514329,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,140.0,3,3,1.000,218.70
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.155190,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,240.0,40,41,1.025,11703.07
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,0.0,1,1,1.000,158.00
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.000000,1.000000,5.00,0.0,1,1,1.000,79.99
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.000000,0.000000,1.00,100.0,1,1,1.000,167.99


## Step 1

Compute, for each `seller_id`, and cumulated since the beginning:
- The `revenues` it brings
- The `costs` associated with all its bad reviews
- The resulting `profits` (revenues - costs)
- The number of `orders` (it will impact overall IT costs)

### 1. Total Revenue = Sales * 10% + Months_on_olist * 80 BRL

In [124]:
#1.1 months on olist * 80 BRL
sellers['months_on_olist'] = np.ceil((datetime.now() - sellers['date_first_sale']) / np.timedelta64(1, 'M'))\
                            .apply(lambda x: 1 if x == 0 else x) * 80

In [125]:
#1.2 revenue from sales * 10%
sellers['revenue'] = sellers['sales'] * 0.1

In [126]:
#1.3 total revenue (both streams)
sellers['total_revenue'] = sellers['months_on_olist'] + sellers['revenue']

In [127]:
sellers.head()

,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,share_of_one_stars,share_of_five_stars,review_score,review_cost,n_orders,quantity,quantity_per_order,sales,months_on_olist,revenue,total_revenue,profit
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,1.514329,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,0.333333,0.333333,3.00,140.0,3,3,1.000,218.70,3920.0,21.870,3941.870,201.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.155190,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,0.050000,0.725000,4.55,240.0,40,41,1.025,11703.07,4000.0,1170.307,5170.307,2130.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.000000,1.000000,5.00,0.0,1,1,1.000,158.00,2720.0,15.800,2735.800,95.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.000000,1.000000,5.00,0.0,1,1,1.000,79.99,2720.0,7.999,2727.999,87.999
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,1.000000,0.000000,1.00,100.0,1,1,1.000,167.99,3360.0,16.799,3376.799,-3.201


### 2. Review_cost = Mapping of reviews and IT costs

    # modify get_revie_score function

    #def get_review_score(self):
        
        #"""
        #Returns a DataFrame with:
        #'seller_id', 'share_of_one_stars',
        #'share_of_five_stars', 'review_score', 'review_cost'
        #"""
        
        # Since the same seller can appear multiple times in the same order,
        # create a (seller <> order) matching table
        # get and merge data
        
        # matching_table = self.matching_table
        # orders_reviews = self.order.get_review_score()
        # matching_table = matching_table[['order_id', 'seller_id']].drop_duplicates()
        # df = matching_table.merge(orders_reviews, on='order_id')

        # compute the costs
        df['review_cost'] = df['review_score'].map({1: 100,    <--- create a dictionary with costs!
                                                     2: 50,
                                                     3: 40,
                                                     4: 0.,
                                                     5: 0.})
        
        # Groupby seller_id
        df = df.groupby('seller_id', as_index=False).agg({'dim_is_one_star': 'mean',
                                                          'dim_is_five_star': 'mean',
                                                          'review_score': 'mean',
                                                          'review_cost': 'sum'})    <--- create a sum!
        # Rename columns
        df.columns = ['seller_id', 'share_of_one_stars',
                      'share_of_five_stars', 'review_score', 'review_cost']   <--- add a new column for review_cost!

        #return df


In [128]:
sellers[['seller_id', 'review_cost', 'total_revenue']]

,seller_id,review_cost,total_revenue
0,3442f8959a84dea7ee197c632cb2df15,140.0,3941.870
1,d1b65fc7debc3361ea86b5f14c68d2e2,240.0,5170.307
2,ce3ad9de960102d0677a81f5d0bb7b2d,0.0,2735.800
3,c0f3eea2e14555b6faeea3dd58c1b1c3,0.0,2727.999
4,51a04a8a6bdcb23deccc82b0b80742cf,100.0,3376.799
...,...,...,...
2965,98dddbc4601dd4443ca174359b237166,0.0,2735.800
2966,f8201cab383e484733266d1906e2fdfa,140.0,3848.900
2967,74871d19219c7d518d0090283e03c137,0.0,3095.004
2968,e603cf3fec55f8697c9059638d6c8eb5,50.0,4109.700


## 3. Profit per seller

In [129]:
#3. Calculate profit per seller
sellers['profit'] = sellers['total_revenue'] - sellers['review_cost']

In [130]:
sellers[['seller_id', 'review_cost', 'total_revenue', 'profit']].sort_values(by='profit', ascending=False)

,seller_id,review_cost,total_revenue,profit
868,53243585a1d6dc2643021fd1853d8905,5270.0,25877.605,20607.605
67,7e93a43ef30c4f03f38b393420bc753a,4800.0,22043.187,17243.187
534,fa1c13f2614d7b5c4749cbc52fecda94,6750.0,23564.203,16814.203
2153,edb1ef5e36e0c8cd84eb3c9b003e486d,1520.0,11448.455,9928.455
9,ccc4bbb5f32a6ab2b7066a4130f114e3,2220.0,11560.462,9340.462
...,...,...,...,...
315,8b321bb669392f5163d04c59e235e066,15620.0,5193.569,-10426.431
946,ea8482cd71df3c1969d7b9473ff13abc,20710.0,7317.752,-13392.248
1358,cc419e0650a3c5ba77189a1882b7556a,30000.0,14588.842,-15411.158
2360,4a3ca9315b744ce9f8e9374361493884,40890.0,24207.292,-16682.708


## 4. Analyse results

In [131]:
# list of sellers with accumulated loss
neg_sellers = sellers[sellers['profit'] <= 0].sort_values(by='profit', ascending=False)
neg_sellers['profit'].describe()

count       32.000000
mean     -4243.530688
std       5296.818715
min     -19859.517000
25%      -5585.461000
50%      -1933.094000
75%       -779.241500
max        -44.147000
Name: profit, dtype: float64

In [132]:
# accumulated loss associated with sellers with 1,2,3-star reviews
BRL = (neg_sellers['profit'].sum() * -1).round(2)
GBP = (BRL / 7.30).round(2)
print(f"R$ {BRL} is the equivalent to £{GBP}")

R$ 135792.98 is the equivalent to £18601.78


## Step 2
We can then sort sellers by increasing profits for Olist, and for each number of sellers to remove, compute the financial impact it would have made had they never been accepted on the platform. We may find an optimal number of sellers to remove that maximizes Olist's profit margin

In [133]:
positive_sellers = sellers[sellers['profit'] >= 0].sort_values(by='profit', ascending=False)

BRL = (positive_sellers['profit'].sum()).round(2)
GBP = (BRL / 7.30).round(2)
print(f"R${BRL} is the equivalent to £{GBP}")

R$10014118.21 is the equivalent to £1371797.02


In [108]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [134]:
# instanciate a model
model1 = smf.ols(formula='review_cost ~ profit', data=sellers)

# train the model to find best line
model1 = model1.fit() 
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            review_cost   R-squared:                       0.321
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     1403.
Date:                Thu, 06 May 2021   Prob (F-statistic):          9.05e-252
Time:                        17:13:16   Log-Likelihood:                -26125.
No. Observations:                2970   AIC:                         5.225e+04
Df Residuals:                    2968   BIC:                         5.227e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3346.8147     79.935     41.869      0.000    3190.082    3503.547
profit        -0.8372      0.022    -37.454      0.000      -0.881      -0.793
==============================================================================
Omnibus:                     3829.484   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           682842.175
Skew:                           7.092   Prob(JB):                         0.00
Kurtosis:                      75.916   Cond. No.                     9.74e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
# Write down a detailed strategy (step by step) to create the DataFrame you need for step 1
# Think about how to re-use or update the logic you have already coded in your `olist` package

<details>
    <summary>Hints</summary>


Starting from your current `seller().get_training_data()` DataFrame:
- Can you easily transform it to compute Olist's positive `revenue_per_seller`? 
- Can you easily transform it to compute Olist's `cost_of_bad_reviews`?

❓Instead of starting again from scratch, investigate your source code in `seller.py` - how was the mean `review_score` per seller computed? Can you imagine a way to amend your code  to compute `cost_of_reviews` in the same process? 
</details>


## Your turn!

In [20]:
# Keep this notebook tidy, you will present it orally to Olist's CEO at the end of the Communicate topic

In [118]:
!git add ..

In [119]:
!git commit -m 'Completed ceo_request'

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../../02-Statistical-Inference/02-Simple-Analysis/simple_analysis.ipynb
	modified:   ../../03-Linear-Regression/02-Sellers/sellers.ipynb
	modified:   ../../03-Linear-Regression/Optional-Products/products.ipynb
	modified:   ../../olist/order.py
	modified:   ../../olist/product.py
	modified:   ../../olist/seller.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../../olist/order_renato.py
	../../olist/seller_renato.py

no changes added to commit (use "git add" and/or "git commit -a")


In [121]:
!git push origin master

Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 8 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 12.97 KiB | 6.48 MiB/s, done.
Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 3 local objects.
To github.com:boemer00/data-challenges.git
   6525981..71392d9  master -> master
